In [17]:
import pandas as pd
import spacy
from spacy import displacy
from string import punctuation
import stopit
import time

In [2]:
'''
flow
1) load data and spacy
2) run NER
3) identify nouns
4) order nouns ascdening by frequency
5) create list with search terms and preference per row
6) load redirects and article mapping
7) identify best article
8) get category tree
'''

'\nflow\n1) load data and spacy\n2) run NER\n3) identify nouns\n4) order nouns ascdening by frequency\n5) create list with search terms and preference per row\n6) load redirects and article mapping\n7) identify best article\n8) get category tree\n'

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

In [4]:
t_data = pd.read_pickle('C:/Users/Fredi/Kodningsprojekt/data-analysis/workproduct-files/cleaned-dataframes/t_dataMaster-duplicatesRemoved.pkl')


In [5]:
nlp = spacy.load("en_core_web_sm")

In [6]:
spacy.explain("obj")

'object'

In [7]:
text = "Which Egyptian queen was married to king Akhenaten?"

In [8]:
# https://spacy.io/usage/spacy-101
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

for token in doc:
    print(token.text, token.lemma_)

Which which
Egyptian egyptian
queen queen
was be
married married
to to
king king
Akhenaten Akhenaten
? ?


In [9]:
#https://spacy.io/api/annotation#pos-tagging

In [10]:
doc = nlp(text)
for token in doc:
    print(token.text, token.pos_, token.dep_, token.is_stop, token.lemma_)
displacy.render(doc, style="dep")

Which DET det True which
Egyptian ADJ amod False egyptian
queen NOUN nsubj False queen
was AUX ROOT True be
married ADJ acomp False married
to ADP prep True to
king NOUN pobj False king
Akhenaten PROPN pobj False Akhenaten
? PUNCT punct False ?


In [11]:
@stopit.threading_timeoutable(default='not finished')
def namedEntities(a):
    doc = nlp(a)
    toReturn = []
    for ent in doc.ents:
        toReturn.append((ent.text, ent.label_))

    return toReturn

In [12]:
def getSubjects(a):
    doc = nlp(a.lower())
    toReturn = []
    dep_tag = ['csubj', 'nsubj']
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        if(token.dep_ in dep_tag):
            toReturn.append(token.lemma_)
    
    return toReturn

In [13]:
def getObjects(a):
    doc = nlp(a.lower())
    toReturn = []
    dep_tag = ['iobj', 'obj', 'dobj', 'pobj']
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        if(token.dep_ in dep_tag):
            toReturn.append(token.lemma_)
    
    return toReturn

In [14]:
def getNouns(a):
    #doc = nlp(a.lower())
    doc = nlp(a)
    toReturn = []
    pos_tag = ['NOUN']
    for token in doc:
        #toReturn.append([token.lemma_, token.pos_])
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        if(token.pos_ in pos_tag):
            toReturn.append(token.lemma_)
    
    return toReturn

In [ ]:
getNouns("Which composer wrote the classical piece Messiah?")

In [ ]:
questions = pd.DataFrame(t_data.loc[:, "CONS_question"])

In [ ]:
%%time
questions["namedEntities"] = questions["CONS_question"].apply(lambda x: namedEntities(x, timeout=2))

In [18]:
@stopit.threading_timeoutable(default='not finished')
def bar(a):
    for i in range(a):
        print("Tick")
        time.sleep(1)

In [21]:
bar(5, timeout=3)

Tick
Tick
Tick
Tick

'not finished'

In [ ]:
questions["subjects"] = questions["CONS_question"].apply(lambda x: getSubjects(x))

In [ ]:
questions["objects"] = questions["CONS_question"].apply(lambda x: getObjects(x))

In [ ]:
questions["nouns"] = questions["CONS_question"].apply(lambda x: getNouns(x))

In [ ]:
questions[0:10]

In [ ]:
word_frequencies = pd.read_csv("F:/Word frequencies/unigram_freq.csv")

In [ ]:
questions.iloc[0:2,-1]

In [ ]:
to_find1 = ["state", "cheyenne", "capital", "city"]
to_find = questions.iloc[1,-1]
word_frequencies[word_frequencies["word"].isin(to_find1)]

In [ ]:
a = questions.iloc[0:2,-1].apply(lambda x: word_frequencies[word_frequencies["word"].isin(x)])
a

In [ ]:
def frequencyCount(a):
    result = []
    for word in a:
        try:
            result.append(word_frequencies.loc[word_frequencies["word"] == word, "count"].iloc[0])
        except IndexError:
            result.append("noValueFound")
    return result    
    

In [ ]:
%%time
questions["nounFrequencies"] = questions["nouns"].apply(lambda x: frequencyCount(x))

In [ ]:
questions[470:500]

In [ ]:
values = questions.iloc[3,-1]
values.index(min(values))

In [ ]:
test = frequencyCount(questions.iloc[1,-1])

In [ ]:
questions.iloc[1,-1]

In [ ]:
test

In [ ]:
word_frequencies.loc[word_frequencies["word"] == "alskdugjasökdg", "count"].iloc[0]

In [ ]:
word_frequencies.iloc[0,1]

In [ ]:
#t_data.CONS_category.unique()

In [ ]:
#t_data.loc[22502]